In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


##### adding watermark #####
import pickle
from pathlib import Path
import argparse
import numpy as np
import os
import os.path as osp
import random
import time
from tqdm import tqdm

from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import chi2

def getGreenList(lo=0, hi=1, m=1000):
    """ return a list of tuple, representing the green list
    """

    waymarks = np.linspace(lo, hi, m + 1)

    green_list = []
    for i in range(0, m, 2):
        # select $delta_i^{2k}$ in equ.(2) with prob.
        if np.random.uniform() > .5:
            i += 1
        green_list.append([waymarks[i], waymarks[i + 1]])

    return green_list

def singleColumnWatermark(arr, GreenListsInfo, col_id, m):
    """ add watermark to a column based on a green list, normalized by the
        empirical statistics of a generative model
    """
    arr_wm = arr.copy()
    mean = GreenListsInfo['col_mean'][col_id]
    std = GreenListsInfo['col_std'][col_id]
    green_list = GreenListsInfo['GreenLists'][col_id]
    arr_wm = (arr_wm - mean) / std
    
    for i in range(len(arr_wm)):
        # offset elem to [0, 1]
        e_flr = np.floor(arr_wm[i])
        e = arr_wm[i] - e_flr

        g = findNearestIntervalHash(e, green_list, m)

        if e > g[1] or e < g[0]:
            # if x[i] falls outside of the range, then
            # we re-sample the elem. from a uniform dist.
            e_wm = np.random.uniform(g[0], g[1]) 
            arr_wm[i] = e_wm + e_flr

        else:
            e_wm = e
    
        # print("elem: {}, int: {}, {}, elem_wm: {}, ofst: {}".format(e, g[0], g[1], e_wm, e_flr))
    arr_wm = arr_wm * std + mean

    return arr_wm, mean, std

def findNearestInterval(e, green_list):
    """ return the nearest interval to the given element e
    """

    min_dist, min_indx = np.inf, -1

    # offset elem to [0, 1]
    e = e - np.floor(e)

    for i, intv in enumerate(green_list):
        cur_dist = np.abs(e - (intv[0] + intv[1]) / 2)

        if cur_dist < min_dist:
            min_dist = cur_dist
            min_indx = i

    return green_list[min_indx]

def findNearestIntervalHash(e, green_list, m):
    """ return the nearest interval to the given element e
    """

    min_dist, min_indx = np.inf, -1

    # offset elem to [0, 1]
    e = e - np.floor(e)

    # hashing
    idx_c = int(e // (2 / m))
    idx_l0, idx_r0 = max(0, idx_c - 1), min(idx_c + 1, len(green_list) - 1)
    idx_l1, idx_r1 = max(0, idx_c - 2), min(idx_c + 2, len(green_list) - 1)

    local_g_list = [green_list[idx_l1], green_list[idx_l0], green_list[idx_c], green_list[idx_r0], green_list[idx_r1]]
    for i, intv in enumerate(local_g_list):
        cur_dist = np.abs(e - (intv[0] + intv[1]) / 2)

        if cur_dist < min_dist:
            min_dist = cur_dist
            min_indx = i

    return local_g_list[min_indx]

def countElemInGreenList(arr, green_list, m):
    """ Count elements in the array falling within the green list intervals.
    """
    T = 0

    for e in arr:
        g = findNearestIntervalHash(e, green_list, m)

        e_ofst = e - np.floor(e)

        T += (e_ofst <= g[1] and e_ofst >= g[0])
        # print(g[0], g[1], e, e_ofst)

    return T

def binomTesting(arr, green_list, m):
    T = countElemInGreenList(arr, green_list, m)
    p = binom.cdf(T, len(arr), 0.5)
    print(T)
    return 1 - p

def gaussTesting(arr, green_list, m):
    T = countElemInGreenList(arr, green_list, m)

    mv = len(arr) / 2
    std = np.sqrt(len(arr)) / 2
    p = norm.cdf((T - mv) / std)
    print(T)
    return 1 - p    

def gaussMultTesting(arrs, green_lists, m):
    """ The proposed method which performs hypothesis testing across multiple arrays with chi-squared distribution.
    """
    X = 0

    mv = len(arrs[0]) / 2
    std = np.sqrt(len(arrs[0])) / 2

    for arr, green_list in zip(arrs, green_lists):
        T = countElemInGreenList(arr, green_list, m)
        #print(T)

        T_c = (T - mv) / std
        X += T_c ** 2

    p = chi2.cdf(X, len(arrs))
    return 1 - p

def exactMaxGaussCDF(x, n):
    return norm.cdf(x)**n

def maxGaussTesting(arrs, green_lists, m):
    """ an optional hypothesis testing method with extreme statistics
    """
    max_Tc = float('-inf')

    mv = len(arrs[0]) / 2
    std = np.sqrt(len(arrs[0])) / 2

    for arr, green_list in zip(arrs, green_lists):
        T = countElemInGreenList(arr, green_list, m)
        T_c = (T - mv) / std
        if T_c > max_Tc:
            max_Tc = T_c

    p = exactMaxGaussCDF(max_Tc, len(arrs))
    return 1 - p

def genMixtureGaussData(n, k=5):
    mv = np.random.randn(1, k) + np.random.randn(n, k)
    ma = np.eye(k)[np.random.choice(k, n)]
    return np.sum(mv * ma, axis=1)

def simulateData(n=10, m=1000):
    tab_list_wm = [np.random.randn(n) for __ in range(5)]
    tab_list_nwm = [np.random.randn(n) for __ in range(5)]
    # create green list
    g_list = getGreenList(m=m)

    print("############# n = {} #############".format(n))
    for tab in tab_list_nwm:
        print("Binom: {:.8f}, Gauss: {:.8f}".format(binomTesting(tab, g_list, m), gaussTesting(tab, g_list, m)))

    for tab in tab_list_wm:
        tab_wm = singleColumnWatermark(tab, g_list, m)
        print("Binom: {:.8f}, Gauss: {:.8f}".format(binomTesting(tab_wm, g_list, m), gaussTesting(tab_wm, g_list, m)))

def simulateMultiColData(n=10, c=10, m=1000):
    # create green list
    g_lists = [getGreenList(m=m) for __ in range(c)]

    tab_lists_no_wm = [np.random.randn(n) for __ in range(c)]
    tab_lists_wm_sing = [np.random.randn(n) for __ in range(c)]
    tab_lists_wm_all = [np.random.randn(n) for __ in range(c)] 

    print("############# n = {}, c = {} #############".format(n, c))
    print("no watermark w/ chi2 testing: {:.8f}".format(gaussMultTesting(tab_lists_no_wm, g_lists, m)))

    tab_lists_wm_sing[0] = singleColumnWatermark(tab_lists_wm_sing[0], g_lists[0], m)
    print("Single-column watermark w/ chi2 testing: {:.8f}".format(gaussMultTesting(tab_lists_wm_sing, g_lists, m)))

    for i, tab in enumerate(tab_lists_wm_all):
        tab_lists_wm_all[i] = singleColumnWatermark(tab, g_lists[i], m)
    print("All-column watermark w/ chi2 testing: {:.8f}".format(gaussMultTesting(tab_lists_wm_all, g_lists, m)))


# In[2]:


def singleColumnNoise(arr, noise_prop, std_prop):
    """ add gaussian noise to a column
    """
    arr_noise = arr.copy()
    
    num_rows_to_modify = int(len(arr) * noise_prop)
    noise_std = np.sqrt(np.var(arr)) * std_prop
    noise = np.random.normal(0, noise_std, num_rows_to_modify)
    
    # add noise to top x% rows
    arr_noise[:num_rows_to_modify] += noise
    
    return arr_noise


# In[ ]:


def get_optimal_col_and_m(X_num, delta=0.01):
    '''  Empirical method to filter out poorly distributed numerical columns
    '''
    dataset  = X_num.copy()
    
    for col_id in range(dataset.shape[1]):
        dataset[:, col_id] = (dataset[:, col_id] - np.mean(dataset[:, col_id])) / np.std(dataset[:, col_id])
    
    prop_min = 0.5 - delta
    prop_max = 0.5 + delta
    
    m_list = list(range(1000, 5001, 500))
    filter_arr = np.zeros(dataset.shape[1])
    n = dataset.shape[0]
    
    test_times = 5
    
    for time in range(test_times):
        print('test:', time)
        for m in m_list:
            print('m:', m)
            unit_filter_arr = np.zeros(dataset.shape[1])
            
            for col_id in range(dataset.shape[1]):
                arr = dataset[:, col_id]
                col_GreenList = getGreenList(m=m)
                prop_col = countElemInGreenList(arr, col_GreenList, m=m) / n
                
                if prop_col < prop_min or prop_col > prop_max:
                    unit_filter_arr[col_id] += 1
            
            filter_arr = np.vstack((filter_arr, unit_filter_arr))
    
    filter_arr = filter_arr[1:, :]
    filter_continuous = np.where(np.sum(filter_arr, axis=0)/(test_times*len(m_list)) < 0.1)[0]
        
    if not len(filter_continuous):
        return np.array([]), None
    
    else:
        filter_arr = filter_arr[:, filter_continuous]
        print('filter_continuous:', filter_continuous, len(filter_continuous))
        
        agg_by_m = np.array([])

        if len(filter_continuous):
            
            for step in range(len(m_list)):
                agg_by_m = np.append(agg_by_m, filter_arr[step::len(m_list)].sum())
            
            print('pass_times', agg_by_m)
            
            optimal_m = m_list[np.argmin(agg_by_m)]
        
        else:
            return filter_continuous, None
        
        print('optimal_m:', optimal_m)
        
    return filter_continuous, optimal_m

def get_p_value(file_path, file_index):
    ''' get p-value for (i) un-watermarked generated data; (ii) watermarked generated data; (iii) watermarked generated data with noise.
        There should be 
        1 file GreenListsInfo.pkl documented greenlist informatiom
        3 corresponding files (i) X_num_train_raw_syn_{file_index}.npy (ii)X_num_train_marked_syn_{file_index}.npy and (iii) X_num_train_noised_syn_{file_index}.npy under the file_path.
    '''

    sub_dir_path = Path(file_path)
    with open(os.path.join(sub_dir_path, f'GreenListsInfo.pkl'), 'rb') as f:
        GreenListsInfo = pickle.load(f)

    continuous_indexes = GreenListsInfo['filter_continuous']
    GreenLists = [GreenListsInfo['GreenLists'][i] for i in continuous_indexes]
    col_mean = np.array(GreenListsInfo['col_mean'])
    col_std = np.array(GreenListsInfo['col_std'])
    
    
    raw_data = np.load(sub_dir_path / f'X_num_train_raw_syn_{file_index}.npy')
    raw_data = (raw_data - col_mean) / col_std
    raw_data = raw_data[:, continuous_indexes]
    p_raw = maxGaussTesting(raw_data.T, GreenLists, m=len(GreenLists[0])*2)
    
    watermarked_data = np.load(sub_dir_path / f'X_num_train_marked_syn_{file_index}.npy')
    watermarked_data = (watermarked_data - col_mean) / col_std
    watermarked_data = watermarked_data[:, continuous_indexes]
    p_watermarked = maxGaussTesting(watermarked_data.T, GreenLists, m=len(GreenLists[0])*2)

    noised_data = np.load(sub_dir_path / f'X_num_train_noised_syn_{file_index}.npy')
    noised_data = (noised_data - col_mean) / col_std
    noised_data = noised_data[:, continuous_indexes]
    p_noised = maxGaussTesting(noised_data.T, GreenLists, m=len(GreenLists[0])*2)
    return [p_raw, p_watermarked, p_noised]

In [2]:
from sklearn.metrics import roc_auc_score

# experiment codes
# organize your folder as
# base_path / task / X_num_train_raw_syn_{i}.npy



GreenLists = []
base_path = Path(r'C:\watermark_exp_test')
num_files = 10 # Total number of raw data files generated by the model
num_sample_data = 5 # Number of sample data files used to estimate empirical distribution

# Ensure your folder is organized as: base_path / task / X_num_train_raw_syn_{i}.npy

for task in ['california']:
             #, 'diabetes', 'gesture', 'house', 'wilt', 'higgs-small', 'miniboone']:
    
    sub_dir_path = base_path / task
    
    GreenLists = []
    GreenListsInfo = {'GreenLists':[], 'col_mean':[], 'col_std':[], 'filter_continuous':[], 'm':[]}
    
    # Step 1: Read all raw generated data files
    X = []
    for i in range(num_files):
        X.append(np.load(os.path.join(sub_dir_path, f'X_num_train_raw_syn_{i}.npy')))
    
    # Step 2: Combine sample data files to form an empirical distribution
    # Use the first `num_sample_data` files for this purpose
    X_arr = []
    for i in range(num_sample_data):
        X_arr.append(X[i])
    X_num = np.concatenate(X_arr, axis=0)

    # Step 3: Identify valid columns and the optimal value of `m`
    filter_continuous, m = get_optimal_col_and_m(X_num)
    
    if len(filter_continuous):
        # Step 4: Generate green lists and compute statistics for filtered columns
        for col_id in range(X_num.shape[1]):
            col_GreenList = getGreenList(m=m)
            GreenLists.append(col_GreenList)
            col_mean, col_std = np.mean(X_num[:, col_id]), np.std(X_num[:, col_id])
            GreenListsInfo['col_mean'].append(col_mean)
            GreenListsInfo['col_std'].append(col_std)

        # Store green list information 
        GreenListsInfo['GreenLists'] = GreenLists
        GreenListsInfo['filter_continuous'] = filter_continuous
        GreenListsInfo['m'] = m

        print('original_features:', X_num.shape[1])
        print('good_features:', len(filter_continuous))
        
        with open(os.path.join(sub_dir_path, f'GreenListsInfo.pkl'), 'wb') as f:
            pickle.dump(GreenListsInfo, f)
        print('GreenList is successfully generated and saved.')

        # Step 5: Apply watermarks and save modified files
        for i in range(num_files):

            if i % 10 == 0:
                print(i)
            
            X_num = X[i]
            #np.save(os.path.join(sub_dir_path, str(i), f'X_num_train_raw_syn_{i}.npy'), X_num)
            
            for col_id in range(X_num.shape[1]):
                col_arr = X_num[:, col_id]
                col_arr_wm, col_mean, col_std = singleColumnWatermark(X_num[:, col_id], GreenListsInfo, col_id, m=m)
                X_num[:, col_id] = col_arr_wm

            np.save(os.path.join(sub_dir_path, f'X_num_train_marked_syn_{i}.npy'), X_num)

            # Add Gaussian noise to each column
            for col_id in range(X_num.shape[1]):
                col_arr = X_num[:, col_id]
                col_arr_noise = singleColumnNoise(col_arr, 0.95, 0.01)
                X_num[:, col_id] = col_arr_noise

            np.save(os.path.join(sub_dir_path, f'X_num_train_noised_syn_{i}.npy'), X_num)


        print('-------------------------')
        print(f'Finished processing task: {task}')

        # Step 6: Compute AUC scores

        p_raw_list = []
        p_wm_list = []
        p_noised_list = []

        y = np.concatenate([np.ones(num_files), np.zeros(num_files)])
        for i in range(num_files):
            #print(filter_continuous)
            p_raw, p_watermarked, p_noised = get_p_value(file_path = sub_dir_path, file_index = i)
            p_raw_list.append(1 - p_raw)
            p_wm_list.append(1 - p_watermarked)
            p_noised_list.append(1 - p_noised)

        print('original auc', roc_auc_score(y, np.array(p_wm_list + p_raw_list)))
        print('noised auc', roc_auc_score(y, np.array(p_noised_list + p_raw_list)))

test: 0
m: 1000
m: 1500
m: 2000
m: 2500
m: 3000
m: 3500
m: 4000
m: 4500
m: 5000
test: 1
m: 1000
m: 1500
m: 2000
m: 2500
m: 3000
m: 3500
m: 4000
m: 4500
m: 5000
test: 2
m: 1000
m: 1500
m: 2000
m: 2500
m: 3000
m: 3500
m: 4000
m: 4500
m: 5000
test: 3
m: 1000
m: 1500
m: 2000
m: 2500
m: 3000
m: 3500
m: 4000
m: 4500
m: 5000
test: 4
m: 1000
m: 1500
m: 2000
m: 2500
m: 3000
m: 3500
m: 4000
m: 4500
m: 5000
filter_continuous: [0 2 3 4 5] 5
pass_times [0. 0. 0. 0. 1. 0. 0. 0. 0.]
optimal_m: 1000
original_features: 8
good_features: 5
GreenList is successfully generated and saved.
0
-------------------------
Finished processing task: california
original auc 1.0
noised auc 1.0


In [3]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import jensenshannon
from scipy.stats import wasserstein_distance
from tqdm import tqdm
##### adding watermark #####
import pickle
from pathlib import Path
import argparse
import numpy as np
import os
import os.path as osp
import random
import time
from tqdm import tqdm

from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import chi2

# data_path saving the original data
data_path = Path(r'C:\data')
# base_path saving generated data (watermarked / unwatermarked / noised)
base_path = Path(r'C:\watermark_exp')
num_files = 50

def normalize_array(x):
    for col_id in range(x.shape[1]):
        x[:, col_id] = (x[:, col_id] - np.mean(x[:, col_id])) / np.std(x[:, col_id])

for task in ['california', 'gesture', 'house', 'wilt', 'higgs-small', 'miniboone']:
    print('-----------------------')
    print(task)
    original = np.load(data_path / task / 'X_num_train.npy')
    distance_raw_ori = []
    distance_marked_raw = []
    distance_noised_marked = []
    
    for i in tqdm(num_files):
        raw_syn = np.load(base_path / task / f'X_num_train_raw_syn_{i}.npy')
        marked_syn = np.load(base_path / task / f'X_num_train_marked_syn_{i}.npy')
        noised_syn = np.load(base_path / task / f'X_num_train_noised_syn_{i}.npy')

        normalize_array(original)
        normalize_array(raw_syn)
        normalize_array(marked_syn)
        normalize_array(noised_syn)

        unit_distance_raw_ori = []
        unit_distance_marked_raw = []
        unit_distance_noised_marked = []

        for col_id in range(original.shape[1]):
            unit_distance_raw_ori.append(wasserstein_distance(original[:,col_id], raw_syn[:,col_id]))
            unit_distance_marked_raw.append(wasserstein_distance(marked_syn[:,col_id], raw_syn[:,col_id]))
            unit_distance_noised_marked.append(wasserstein_distance(marked_syn[:,col_id], noised_syn[:,col_id]))
            
        distance_raw_ori.append(sum(unit_distance_raw_ori) / len(unit_distance_raw_ori))
        distance_marked_raw.append(sum(unit_distance_marked_raw) / len(unit_distance_marked_raw))
        distance_noised_marked.append(sum(unit_distance_noised_marked) / len(unit_distance_noised_marked))
    print('wasserstein_distance between original data and synthetic data:', sum(distance_raw_ori)/len(distance_raw_ori))
    print('wasserstein_distance between synthetic data and watermarked data:', sum(distance_marked_raw)/len(distance_marked_raw))
    print('wasserstein_distance between watermarked data and noised data:', sum(distance_noised_marked)/len(distance_noised_marked))

-----------------------
california


100%|██████████| 50/50 [00:08<00:00,  6.11it/s]


wasserstein_distance between original data and synthetic data: 0.022215891410705662
wasserstein_distance between synthetic data and watermarked data: 0.000268862759779327
wasserstein_distance between watermarked data and noised data: 0.0019194950523449125
-----------------------
gesture


100%|██████████| 50/50 [00:14<00:00,  3.48it/s]


wasserstein_distance between original data and synthetic data: 0.060179562425967205
wasserstein_distance between synthetic data and watermarked data: 0.0002658256081741967
wasserstein_distance between watermarked data and noised data: 0.0018698861508541851
-----------------------
house


...

wasserstein_distance between original data and synthetic data: 0.07672873875226098
wasserstein_distance between synthetic data and watermarked data: 0.00044484284926277673
wasserstein_distance between watermarked data and noised data: 0.002184707995364442
-----------------------
higgs-small


100%|██████████| 50/50 [02:33<00:00,  3.07s/it]


wasserstein_distance between original data and synthetic data: 0.014216595430518914
wasserstein_distance between synthetic data and watermarked data: 0.0003442360305712603
wasserstein_distance between watermarked data and noised data: 0.0017334297816506618
-----------------------
miniboone


100%|██████████| 50/50 [06:07<00:00,  7.34s/it]

wasserstein_distance between original data and synthetic data: 0.01610096401723159
wasserstein_distance between synthetic data and watermarked data: 0.00014663095409377783
wasserstein_distance between watermarked data and noised data: 0.003926176236255854
